In [2]:
import numpy as np
import math
import os.path
import csv
import glob
import tensorflow as tf
import h5py as h5py
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model, load_model
import imageio
import matplotlib.pyplot as plt
from mlxtend.image import extract_face_landmarks

ModuleNotFoundError: No module named 'numpy'

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
def mouth_over_eye(eye):
    ear = eye_aspect_ratio(eye)
    mar = mouth_aspect_ratio(eye)
    mouth_eye = mar/ear
    return mouth_eye

In [ ]:
def mouth_aspect_ratio(mouth):
    A = distance.euclidean(mouth[14], mouth[18])
    C = distance.euclidean(mouth[12], mouth[16])
    mar = (A ) / (C)
    return mar

In [ ]:
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

In [ ]:
def circularity(eye):
    A = distance.euclidean(eye[1], eye[4])
    radius  = A/2.0
    Area = math.pi * (radius ** 2)
    p = 0
    p += distance.euclidean(eye[0], eye[1])
    p += distance.euclidean(eye[1], eye[2])
    p += distance.euclidean(eye[2], eye[3])
    p += distance.euclidean(eye[3], eye[4])
    p += distance.euclidean(eye[4], eye[5])
    p += distance.euclidean(eye[5], eye[0])
    return 4 * math.pi * Area /(p**2)

In [ ]:
def getFrame(sec):
    start = 180000
    vidcap.set(cv2.CAP_PROP_POS_MSEC, start + sec*1000)
    hasFrames,image = vidcap.read()
    return hasFrames, image

In [ ]:
from scipy.spatial import distance
import cv2
data = []
labels = []
for j in [60]:
  for i in [10]:
    vidcap = cv2.VideoCapture('drive/My Drive/Fold5_part2/' + str(j) +'/' + str(i) + '.mp4')
    sec = 0
    frameRate = 1
    success, image  = getFrame(sec)
    count = 0
    while success and count < 240: 
          landmarks = extract_face_landmarks(image)
          if sum(sum(landmarks)) != 0:
              count += 1
              data.append(landmarks)
              labels.append([i])
              sec = sec + frameRate
              sec = round(sec, 2)
              success, image = getFrame(sec)
              print(count)
          else:  
              sec = sec + frameRate
              sec = round(sec, 2)
              success, image = getFrame(sec)
              print("not detected")

In [ ]:
import numpy as np
data = np.array(data)
labels = np.array(labels)

features = []
for d in data:
  eye = d[36:68]
  ear = eye_aspect_ratio(eye)
  mar = mouth_aspect_ratio(eye)
  cir = circularity(eye)
  mouth_eye = mouth_over_eye(eye)
  features.append([ear, mar, cir, mouth_eye])

In [ ]:
features = np.array(features)
features.shape

In [ ]:
data.shape
labels.shape

In [ ]:
# change file name to save 
np.save(open('Data_60_10.npy', 'wb'),data)
np.save(open('Fold5_part2_features_60_10.npy', 'wb'),features)
np.save(open('Fold5_part2_labels_60_10.npy', 'wb'),labels)
np.savetxt("Fold5_part2_features_60_10.csv", features, delimiter = ",")
np.savetxt("Fold5_part2_labels_60_10.csv", labels, delimiter = ",")

In [ ]:
# how to load npy data file to colab
a = np.load('Data_32_0.npy')
b = np.load('Fold3_part2_features_32_0.npy')
c = np.load('Fold3_part2_labels_32_0.npy')